# **Section 1: `Load Modules`**

In [17]:
%matplotlib ipympl
from scipy.ndimage import gaussian_filter
import xanes_regtools as xr
import xanes_image_utils as xiu
import matplotlib.pyplot as plt
import h5py, tifffile, sys, os, shutil
import numpy as np
import skimage.morphology as skm
import scipy.ndimage as spn
import SimpleITK as sitk
from copy import deepcopy
from pathlib import Path
from importlib import reload
xr = reload(xr)
xiu = reload(xiu)
plt.rcParams['figure.figsize'] = [14, 8]

import scyjava_config
#scyjava_config.add_options('-Xmx20g')
import imagej
try:
    ij = imagej.init('/home/xiao/software/Fiji.app', headless=False)
    ijui = ij.ui()
    ijui.showUI()
except:
    print('ImageJ is already up running!')
    
from jnius import autoclass
WindowManager = autoclass('ij.WindowManager')

def fiji_close_all(ips=None):
    if ips is None:
        imp = WindowManager.getCurrentImage()
        while imp is not None:
            imp.close()
            imp = WindowManager.getCurrentImage()
    else:
        for ip in ips:
            ip.close()
print('\033[04m\033[01m\033[34m     Section 1 finished     ') 

ImageJ is already up running!
     Section 1 finished     


# **Section 2: `User Inputs`**

In [18]:
# raw h5 file top directory
raw_h5_top_dir = '/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/' 
# recon_top_dir is where the recon folders are
recon_top_dir = '/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/' 
# trial registration save file name
trial_reg_save_file = 'xanes3D_trial_reg_results_29773_new.h5'
# these are [y_start, y_end, x_start, x_end]
roi = [370, 370+440, 660, 660+240]
# alignment section by section; sometime, you may see an abrupt misalignment between two chunks. You may try 
# different chunk_sz to correct such misalignment.
chunk_sz = 7
# scan id start
scan_id_s = 29773
# scan_id_end
scan_id_e = 29873
# reference scan
fixed_scan_id = 29823
# use_anchor: if fixed_scan will be used as an anchor for entire serires alignment
use_anchor = True

# if using a mask in alignment calculation
use_mask = True
# threshold in making a mask
mask_thres = 2e-3
# dilation width for enlarge the mask
mask_dilation = 10

# the index of the reconstructed slice in the reconstruction of the scan with fixed_scan_id
fixed_sli = 430
# a range of slice indices of the reconstructed slices in the reconstruction of 
# the scan with moving_scan_id
moving_sli_search_half_range = 30

# 'sr','pc', 'mpc'  
reg_method = 'mpc'
# ref_mode can be 'single', or 'neighbor' 
ref_mode = 'single'

trial_reg = None
print('\033[04m\033[01m\033[34m     Section 2 finished     ')   

     Section 2 finished     


# **Section 3: `Calculate Displacement Amongst 3D Datasets`**

In [19]:
moving_sli_search_s = fixed_sli - moving_sli_search_half_range
moving_sli_search_e = fixed_sli + moving_sli_search_half_range

recon_path_template = recon_top_dir + 'recon_fly_scan_id_{0}/recon_fly_scan_id_{0}_{1}.tiff'
ffn = recon_top_dir + 'recon_fly_scan_id_'+ str(fixed_scan_id) + \
      '/recon_fly_scan_id_' + str(fixed_scan_id) + '_' + str(fixed_sli).zfill(5) + '.tiff'
fixed  = tifffile.imread(ffn)[roi[0]:roi[1], roi[2]:roi[3]]
moving = np.ndarray([2*moving_sli_search_half_range, fixed.shape[0], fixed.shape[1]])

if use_mask:
    mask = skm.binary_dilation((fixed > mask_thres).astype(np.uint8),
                               np.ones([mask_dilation,mask_dilation])).astype(np.uint8)
    ijui.show(ij.py.to_java(mask))
    ijui.show(ij.py.to_java(fixed))
    ijui.show(ij.py.to_java(fixed*mask))

In [20]:
fiji_close_all()

In [21]:
reg = xr.regtools(dtype='3D_XANES', method=reg_method, mode='TRANSLATION')
reg.set_method(reg_method)
reg.set_ref_mode(ref_mode)
reg.cal_set_anchor(scan_id_s, scan_id_e, fixed_scan_id, raw_h5_top_dir=raw_h5_top_dir)
reg.set_chunk_sz(chunk_sz)
reg.set_roi(roi)
if use_mask:
    reg.use_mask = True
    reg.set_mask(mask)
reg.set_fixed_data(fixed)
reg.set_3D_recon_path_template(recon_path_template)
reg.set_saving(save_path=recon_top_dir, fn=trial_reg_save_file)
reg.xanes3D_sli_search_half_range = moving_sli_search_half_range
reg.xanes3D_recon_fixed_sli = fixed_sli
reg.reg_xanes3D_chunk()
print('\033[04m\033[01m\033[34m     Section 3 finished     ') 

/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/
The registration results will be saved in /media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/xanes3D_trial_reg_results_29773_new.h5
MPC
We are using "masked phase correlation" method for registration.
[-57. -26.]
[-60. -27.]
[-60. -28.]
[-60. -29.]
[-60. -29.]
[-66. -23.]
[-65. -25.]
[-65. -25.]
[-65. -26.]
[-65. -26.]
[-65. -27.]
[-62. -27.]
[-63. -28.]
[-63. -28.]
[22. -1.]
[21. -1.]
[20. -1.]
[20. -1.]
[19. -1.]
[17. -1.]
[15. -2.]
[15. -2.]
[15. -2.]
[14. -2.]
[14. -3.]
[14. -3.]
[14. -3.]
[14. -4.]
[14. -4.]
[13. -4.]
[13. -4.]
[12. -5.]
[12. -5.]
[11. -5.]
[11. -5.]
[11. -4.]
[11. -4.]
[11. -3.]
[-77.  36.]
[-78.  36.]
[-78.  37.]
[-85.  34.]
[-85.  35.]
[-86.  36.]
[-86.  36.]
[-75.  47.]
[-75.  47.]
[-76.  47.]
[-76.  47.]
[-77.  46.]
[-77.  47.]
[-77.  48.]
[-77.  49.]
[-77.  50.]
[-77.  51.]
[-77.  51.]
[-77.  51.]
[-77.  52.]
[-76.  53.]
[-75.  53.]
[50. 24.]
[-39. -42.]
[-39. -43.]
[-39. -43.]
[-39. -44.]
[-39. -44.]
[-39.

In [14]:
cnt = 0
for ii in reg.alignment_pair:
    print(cnt, '\t', np.array(ii), '\t', scan_id_s+np.array(ii))
    cnt += 1

0 	 [50 43] 	 [29823 29816]
1 	 [43 36] 	 [29816 29809]
2 	 [36 29] 	 [29809 29802]
3 	 [29 22] 	 [29802 29795]
4 	 [22 15] 	 [29795 29788]
5 	 [15  8] 	 [29788 29781]
6 	 [8 1] 	 [29781 29774]
7 	 [50 57] 	 [29823 29830]
8 	 [57 64] 	 [29830 29837]
9 	 [64 71] 	 [29837 29844]
10 	 [71 78] 	 [29844 29851]
11 	 [78 85] 	 [29851 29858]
12 	 [85 92] 	 [29858 29865]
13 	 [92 99] 	 [29865 29872]
14 	 [ 99 100] 	 [29872 29873]
15 	 [1 0] 	 [29774 29773]
16 	 [8 2] 	 [29781 29775]
17 	 [8 3] 	 [29781 29776]
18 	 [8 4] 	 [29781 29777]
19 	 [8 5] 	 [29781 29778]
20 	 [8 6] 	 [29781 29779]
21 	 [8 7] 	 [29781 29780]
22 	 [15  9] 	 [29788 29782]
23 	 [15 10] 	 [29788 29783]
24 	 [15 11] 	 [29788 29784]
25 	 [15 12] 	 [29788 29785]
26 	 [15 13] 	 [29788 29786]
27 	 [15 14] 	 [29788 29787]
28 	 [22 16] 	 [29795 29789]
29 	 [22 17] 	 [29795 29790]
30 	 [22 18] 	 [29795 29791]
31 	 [22 19] 	 [29795 29792]
32 	 [22 20] 	 [29795 29793]
33 	 [22 21] 	 [29795 29794]
34 	 [29 23] 	 [29802 29796]
35 	 [29 

In [23]:
#trial_reg = None
try:
    for ip in ips:
        ip.close()
except:
    pass

if trial_reg is None:
    trial_reg = np.ndarray([2*moving_sli_search_half_range, roi[1]-roi[0], roi[3]-roi[2]])
    
alignment_pair_id = 3
fn = "/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/xanes3D_trial_reg_results_29773_new.h5"
f = h5py.File(fn, 'r')
trial_reg[:] = f['/trial_registration/trial_reg_results/{0}/trial_reg_img{0}'.format(str(alignment_pair_id).zfill(3))][:]
f.close()

ips = []
ijui.show(ij.py.to_java(trial_reg))
imp1 = WindowManager.getCurrentImage()
imp1.setTitle(str(alignment_pair_id).zfill(3))
ips.append(imp1)

In [56]:
fn = '/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/zshift_29773.txt'
with open(fn, 'r') as f:
    a = f.readlines()

b = []
for ii in a:
    if ii.split():
        #print(ii.split()[0])
        b.append([int(ii.split()[0]), int(ii.split()[1])+10])
        
fn = '/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/zshift_29773_new.txt'
np.savetxt(fn, b, fmt='%d')

In [55]:
help(np.savetxt)

Help on function savetxt in module numpy:

savetxt(fname, X, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None)
    Save an array to a text file.
    
    Parameters
    ----------
    fname : filename or file handle
        If the filename ends in ``.gz``, the file is automatically saved in
        compressed gzip format.  `loadtxt` understands gzipped files
        transparently.
    X : 1D or 2D array_like
        Data to be saved to a text file.
    fmt : str or sequence of strs, optional
        A single format (%10.5f), a sequence of formats, or a
        multi-format string, e.g. 'Iteration %d -- %10.5f', in which
        case `delimiter` is ignored. For complex `X`, the legal options
        for `fmt` are:
    
        * a single specifier, `fmt='%.4e'`, resulting in numbers formatted
          like `' (%s+%sj)' % (fmt, fmt)`
        * a full string specifying every real and imaginary part, e.g.
          `' %.4e %+.4ej %.4e %+.4ej %.4

# **Section 4: `Align 3D Datasets`**

In [1]:
from importlib import reload
import xanes_regtools as xr
xr = reload(xr)
import numpy as np
import h5py, os
shift = np.int32(np.genfromtxt('/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/zshift_29773_new.txt'))
fn = "/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/xanes3D_trial_reg_results_29773_new.h5"
# this is the slice range for alignment after the shift values are determined
moving_sli_start = 480
moving_sli_end = 610

f = h5py.File(fn, 'r')
recon_top_dir = f['/trial_registration/data_directory_info/recon_top_dir'][()] 
recon_path_template = f['/trial_registration/data_directory_info/recon_path_template'][()] 
roi = f['/trial_registration/trial_reg_parameters/slice_roi'][:] 
reg_method = f['/trial_registration/trial_reg_parameters/reg_method'][()].lower()
ref_mode = f['/trial_registration/trial_reg_parameters/reg_ref_mode'][()].lower()
scan_id_s = f['/trial_registration/trial_reg_parameters/scan_ids'][0]
scan_id_e = f['/trial_registration/trial_reg_parameters/scan_ids'][-1]
fixed_scan_id = f['/trial_registration/trial_reg_parameters/fixed_scan_id'][()]
chunk_sz = f['/trial_registration/trial_reg_parameters/chunk_sz'][()]
eng_list = f['/trial_registration/trial_reg_parameters/eng_list'][:]
f.close()

reg = xr.regtools(dtype='3D_XANES', method=reg_method, mode='TRANSLATION')
reg.set_method(reg_method)
reg.set_ref_mode(ref_mode)
reg.cal_set_anchor(scan_id_s, scan_id_e, fixed_scan_id)
reg.eng_list = eng_list

reg.set_chunk_sz(chunk_sz)
reg.set_roi(roi)
reg.set_3D_recon_path_template(recon_path_template)
reg.set_saving(save_path=recon_top_dir, fn=os.path.basename(fn))
reg.apply_xanes3D_chunk_shift(shift, 410, 450, trialfn=fn, savefn=fn)
print('\033[04m\033[01m\033[34m     Section 4 finished     ') 

OSError: /media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/zshift_29773_new.txt not found.

In [6]:
try:
    for ip in ips:
        ip.close()
except:
    pass

fn = "/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/xanes3D_trial_reg_results_29773_new.h5"
sli_idx = 20
f = h5py.File(fn, 'r')
reg_img = f['/registration_results/reg_results/registered_xanes3D'][:, sli_idx, :, :]
f.close()

ips = []
ijui.show(ij.py.to_java(reg_img))
imp1 = WindowManager.getCurrentImage()
imp1.setTitle(str(sli_idx).zfill(3))
ips.append(imp1)

In [7]:
shift = np.int32(np.genfromtxt('/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/zshift_29773.txt'))
print(shift)

[[  0   7]
 [  1  11]
 [  2   9]
 [  3  11]
 [  4   7]
 [  5   7]
 [  6   8]
 [  7  15]
 [  8  11]
 [  9  10]
 [ 10  19]
 [ 11  10]
 [ 12  13]
 [ 13  14]
 [ 14  14]
 [ 15  12]
 [ 16   1]
 [ 17   1]
 [ 18   5]
 [ 19   5]
 [ 20   6]
 [ 21  10]
 [ 22   9]
 [ 23   8]
 [ 24   9]
 [ 25  10]
 [ 26  11]
 [ 27  10]
 [ 28   8]
 [ 29   9]
 [ 30  11]
 [ 31  11]
 [ 32   9]
 [ 33  11]
 [ 34  10]
 [ 35  12]
 [ 36  10]
 [ 37  11]
 [ 38  11]
 [ 39  10]
 [ 40   9]
 [ 41   8]
 [ 42   8]
 [ 43   9]
 [ 44   8]
 [ 45  11]
 [ 46  12]
 [ 47  11]
 [ 48  12]
 [ 49  10]
 [ 50  12]
 [ 51  11]
 [ 52   6]
 [ 53  10]
 [ 54  11]
 [ 55  10]
 [ 56  10]
 [ 57  11]
 [ 58  12]
 [ 59  11]
 [ 60  11]
 [ 61  11]
 [ 62  12]
 [ 63  11]
 [ 64  11]
 [ 65  12]
 [ 66  11]
 [ 67  11]
 [ 68  11]
 [ 69  11]
 [ 70  11]
 [ 71  11]
 [ 72  11]
 [ 73  11]
 [ 74  11]
 [ 75  11]
 [ 76   2]
 [ 77   2]
 [ 78   2]
 [ 79   2]
 [ 80   7]
 [ 81  11]
 [ 82  12]
 [ 83  11]
 [ 84  11]
 [ 85  11]
 [ 86  11]
 [ 87  11]
 [ 88   8]
 [ 89   9]
 [ 90   9]

In [5]:
print(shift)

[[ 0  7]
 [ 1 11]
 [ 2  9]
 [ 3 11]
 [ 4  7]
 [ 5  7]
 [ 6  8]
 [ 7 15]
 [ 8 11]
 [ 9 10]
 [10 19]
 [11 10]
 [12 13]
 [13 14]
 [14 14]
 [15  1]
 [16  1]
 [17  5]
 [18  5]
 [19  6]
 [20 10]
 [21  9]
 [22  8]
 [23  9]
 [24 10]
 [25 11]
 [26 10]
 [27  8]
 [28  9]
 [29 11]
 [30 11]
 [31  9]
 [32 11]
 [33 10]
 [34 12]
 [35 10]
 [36 11]
 [37 11]
 [38 10]
 [39  9]
 [40  8]
 [41  8]
 [42  9]
 [43  8]
 [44 11]
 [45 12]
 [46 11]
 [47 12]
 [48 10]
 [49 11]
 [50 11]
 [51  6]
 [52 10]
 [53 11]
 [54 10]
 [55 10]
 [56 11]
 [57 12]
 [58 11]
 [59 11]
 [60 11]
 [61 12]
 [62 11]
 [63 11]
 [64 12]
 [65 11]
 [66 11]
 [67 11]
 [68 11]
 [69 11]
 [70 11]
 [71 11]
 [72 11]
 [73 11]
 [74 11]
 [75  2]
 [76  2]
 [77  2]
 [78  2]
 [79  7]
 [80 11]
 [81 12]
 [82 11]
 [83 11]
 [84 11]
 [85 11]
 [86 11]
 [87  8]
 [88  9]
 [89  9]
 [90  8]
 [91  8]
 [92  8]
 [93  9]
 [94 11]
 [95 12]
 [96  7]
 [97  7]
 [98  9]
 [99 12]]


In [3]:
print(reg.eng_list)

[8.21200229 8.22199763 8.23200196 8.24200714 8.2519969  8.26200352
 8.27200259 8.28200213 8.29200205 8.30200225 8.31200265 8.3129983
 8.31400242 8.31499856 8.31600318 8.31699981 8.31799669 8.31900205
 8.31999942 8.32099704 8.32200316 8.32300127 8.32399963 8.32499824
 8.32599709 8.32699619 8.3280038  8.3290034  8.33000324 8.33100333
 8.33200367 8.33300425 8.33399682 8.33499789 8.33599922 8.33699252
 8.33800262 8.33899641 8.33999872 8.34100129 8.3420041  8.34299887
 8.34400218 8.34499744 8.34600124 8.34699699 8.34800129 8.34899753
 8.35000233 8.35099906 8.35198774 8.35300158 8.35399906 8.35499677
 8.35600306 8.35700127 8.35799972 8.35899843 8.35999737 8.36099657
 8.36199601 8.36300403 8.36400396 8.36500414 8.36599623 8.3669969
 8.36799782 8.36899899 8.3700004  8.37100206 8.37200396 8.37299776
 8.37400016 8.3750028  8.37599733 8.37700047 8.37800385 8.37899912
 8.380003   8.38099875 8.38200312 8.38299937 8.38400423 8.38500097
 8.38599794 8.38699517 8.38800102 8.38899873 8.38999669 8.391003

In [18]:
for key, item in reg.img_id_eng_dict.items():
    print(key, item)

29773 8.21200229140451
29774 8.221997632831867
29775 8.232001955056656
29776 8.242007144894401
29777 8.251996902364287
29778 8.262003519828605
29779 8.272002590797916
29780 8.28200213052105
29781 8.292002048249842
29782 8.302002252786103
29783 8.312002652481642
29784 8.312998296868228
29785 8.314002418481964
29786 8.314998556403438
29787 8.31600317581396
29788 8.316999807633604
29789 8.317996685521678
29790 8.319002051288829
29791 8.319999423619919
29792 8.32099704229094
29793 8.322003155238228
29794 8.323001268897915
29795 8.323999629169435
29796 8.32499823614326
29797 8.32599708990991
29798 8.326996190559946
29799 8.328003798284488
29800 8.329003395015395
29801 8.330003238902387
29802 8.33100333003621
29803 8.332003668507644
29804 8.33300425440752
29805 8.333996815461536
29806 8.334997894444221
29807 8.335999221127349
29808 8.336992517111211
29809 8.338002617959003
29810 8.33899640570023
29811 8.339998722045147
29812 8.341001286545229
29813 8.34200409929171
29814 8.342998869596881
29

In [26]:
print(len(reg.abs_shift_dict))
print(len(reg.alignment_pair))
print(len(reg.shift_chain_dict))
cnt = 0
for key, item in reg.abs_shift_dict.items():
    print(cnt, ':\t', key, item)
    cnt += 1

100
101
100
0 :	 100 [-49.   0.]
1 :	 98 [-46.  -3.]
2 :	 97 [-44.  -2.]
3 :	 96 [-44.  -1.]
4 :	 95 [-47.   1.]
5 :	 94 [-42.   0.]
6 :	 93 [-43.  -2.]
7 :	 91 [-42.   1.]
8 :	 90 [-40.   0.]
9 :	 89 [-39.   0.]
10 :	 88 [-39.   1.]
11 :	 87 [-38.   0.]
12 :	 86 [-37.   1.]
13 :	 84 [-36.   2.]
14 :	 83 [-34.   1.]
15 :	 82 [-35.   0.]
16 :	 81 [-34.   1.]
17 :	 80 [-32.   0.]
18 :	 79 [-31.   0.]
19 :	 77 [-29.   0.]
20 :	 76 [-27.   0.]
21 :	 75 [-27.  -2.]
22 :	 74 [-27.  -1.]
23 :	 73 [-25.  -1.]
24 :	 72 [-24.   0.]
25 :	 70 [-22.   0.]
26 :	 69 [-19.  -1.]
27 :	 68 [-18.  -1.]
28 :	 67 [-17.   1.]
29 :	 66 [-16.   1.]
30 :	 65 [-15.   1.]
31 :	 63 [-13.   1.]
32 :	 62 [-11.   0.]
33 :	 61 [-10.   0.]
34 :	 60 [-10.   0.]
35 :	 59 [-9.  0.]
36 :	 58 [-8.  1.]
37 :	 56 [-5.  0.]
38 :	 55 [-4.  0.]
39 :	 54 [-3.  1.]
40 :	 53 [-2.  0.]
41 :	 52 [-2. -2.]
42 :	 51 [ 1. -1.]
43 :	 49 [ 3. -1.]
44 :	 48 [ 7. -1.]
45 :	 47 [ 7. -2.]
46 :	 46 [10. -2.]
47 :	 45 [10. -2.]
48 :	 44 [ 9. -

In [30]:
cnt = 0
for key, item in reg.shift_chain_dict.items():
    print(cnt, ':\t', key, item)
    cnt += 1

0 :	 100 [100, 99, 92, 85, 78, 71, 64, 57, 50]
1 :	 98 [98, 99, 92, 85, 78, 71, 64, 57, 50]
2 :	 97 [97, 99, 92, 85, 78, 71, 64, 57, 50]
3 :	 96 [96, 99, 92, 85, 78, 71, 64, 57, 50]
4 :	 95 [95, 99, 92, 85, 78, 71, 64, 57, 50]
5 :	 94 [94, 99, 92, 85, 78, 71, 64, 57, 50]
6 :	 93 [93, 99, 92, 85, 78, 71, 64, 57, 50]
7 :	 91 [91, 92, 85, 78, 71, 64, 57, 50]
8 :	 90 [90, 92, 85, 78, 71, 64, 57, 50]
9 :	 89 [89, 92, 85, 78, 71, 64, 57, 50]
10 :	 88 [88, 92, 85, 78, 71, 64, 57, 50]
11 :	 87 [87, 92, 85, 78, 71, 64, 57, 50]
12 :	 86 [86, 92, 85, 78, 71, 64, 57, 50]
13 :	 84 [84, 85, 78, 71, 64, 57, 50]
14 :	 83 [83, 85, 78, 71, 64, 57, 50]
15 :	 82 [82, 85, 78, 71, 64, 57, 50]
16 :	 81 [81, 85, 78, 71, 64, 57, 50]
17 :	 80 [80, 85, 78, 71, 64, 57, 50]
18 :	 79 [79, 85, 78, 71, 64, 57, 50]
19 :	 77 [77, 78, 71, 64, 57, 50]
20 :	 76 [76, 78, 71, 64, 57, 50]
21 :	 75 [75, 78, 71, 64, 57, 50]
22 :	 74 [74, 78, 71, 64, 57, 50]
23 :	 73 [73, 78, 71, 64, 57, 50]
24 :	 72 [72, 78, 71, 64, 57, 50]
25

In [32]:
print(len(reg.alignment_pair))
cnt = 0
for key, item in reg.alignment_pair:
    print(cnt, ':\t', key, item)
    cnt += 1

101
0 :	 50 43
1 :	 43 36
2 :	 36 29
3 :	 29 22
4 :	 22 15
5 :	 15 8
6 :	 8 1
7 :	 50 57
8 :	 57 64
9 :	 64 71
10 :	 71 78
11 :	 78 85
12 :	 85 92
13 :	 92 99
14 :	 99 100
15 :	 1 0
16 :	 8 2
17 :	 8 3
18 :	 8 4
19 :	 8 5
20 :	 8 6
21 :	 8 7
22 :	 15 9
23 :	 15 10
24 :	 15 11
25 :	 15 12
26 :	 15 13
27 :	 15 14
28 :	 22 16
29 :	 22 17
30 :	 22 18
31 :	 22 19
32 :	 22 20
33 :	 22 21
34 :	 29 23
35 :	 29 24
36 :	 29 25
37 :	 29 26
38 :	 29 27
39 :	 29 28
40 :	 36 30
41 :	 36 31
42 :	 36 32
43 :	 36 33
44 :	 36 34
45 :	 36 35
46 :	 43 37
47 :	 43 38
48 :	 43 39
49 :	 43 40
50 :	 43 41
51 :	 43 42
52 :	 50 44
53 :	 50 45
54 :	 50 46
55 :	 50 47
56 :	 50 48
57 :	 50 49
58 :	 57 51
59 :	 57 52
60 :	 57 53
61 :	 57 54
62 :	 57 55
63 :	 57 56
64 :	 64 58
65 :	 64 59
66 :	 64 60
67 :	 64 61
68 :	 64 62
69 :	 64 63
70 :	 71 65
71 :	 71 66
72 :	 71 67
73 :	 71 68
74 :	 71 69
75 :	 71 70
76 :	 78 72
77 :	 78 73
78 :	 78 74
79 :	 78 75
80 :	 78 76
81 :	 78 77
82 :	 85 79
83 :	 85 80
84 :	 85 81
85 

In [34]:
print(reg.anchor)

50


In [3]:
int(1.2)

1